# Particle Swarm Optimization

## Environment Setup

### Import Dependencies

In [1]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../..')

from utils.persistency.logger import Logger

from utils.dataset.build_dataset import load_MNIST_data
from utils.dataset.build_dataloader import init_data_loader

from utils.training.train_loop import full_train_loop
from utils.model.model_utils import init_model
from utils.optimization.early_stopper import EarlyStopper
from utils.optimization.regularizer import Regularizer
from utils.misc.device import get_device
from utils.model.model_utils import get_activation_fn, get_loss_fn, get_optimizer
from utils.pso.PSO import PSO
from utils.pso.pso_runner import PSORunner
from utils.pso.PSO import PSOTrial

from utils.display_results.display_results import prediction_loop
from utils.display_results.display_results import display_images
from utils.persistency.file_name_builder import file_name_builder, folder_exists_check

### Init Session

In [2]:
session_num = '000'

In [3]:
outputs_folder_path_csv = 'output_files_PSO/csv'
outputs_folder_path_txt = 'output_files_PSO/txt'

## Load Data

In [4]:
train_dataset, val_dataset, test_dataset = load_MNIST_data('data_MNIST/')

In [5]:
# train_loader = init_data_loader(train_dataset, batch_size=32)
# val_loader = init_data_loader(val_dateset, batch_size=32)
# test_loader = init_data_loader(test_dataset, batch_size=32)

## Optuna Optimization

### Define Objective Function

In [6]:
def objective(trial: PSOTrial, logger: Logger):
    # Define Hyperparameters - Structure HPs
    activation = 'relu'
    num_hidden_layer = round(trial.hyperparameters['num_hidden_layer'])

    network_architecture = [28 * 28]
    for i in range(num_hidden_layer):
        layer_width = round(trial.hyperparameters[f'hidden_layer_n{i+1}_size'])
        if layer_width > 8:
            network_architecture.append(layer_width)
    network_architecture.append(10)
    trial.set_user_attr('network', network_architecture)

    # Define Hyperparameters - Training HPs
    batch_size = 64
    learning_rate = trial.hyperparameters['learning_rate']
    loss_function_str = 'Focal'
    optimizer_str = 'Adam'

    # Define Hyperparameters - Epochs
    max_epochs = 20


    # Init DataLoaders
    train_loader = init_data_loader(train_dataset, batch_size=batch_size)
    val_loader = init_data_loader(val_dataset, batch_size=batch_size)
    test_loader = init_data_loader(test_dataset, batch_size=batch_size)

    # Init Model
    model_extra_args = {'network_architecture': network_architecture, 'activation': get_activation_fn(activation)}
    model = init_model(model_str='MLP', extra_args=model_extra_args).to(get_device())

    # Init Loss
    loss_fn = get_loss_fn(loss_str=loss_function_str)
    
    # Init Optimizer
    optimizer = get_optimizer(model=model, optimizer_str=optimizer_str, learning_rate=learning_rate)

    # Init Regularizer
    regularizer = Regularizer(lambda_tot_widths=0.4, max_depth=3, max_width=128)

    # Init Early Stopper
    early_stopper = EarlyStopper(patience=5, mode="maximize")
    
    
    # Perform Training
    optim_score = full_train_loop(max_epochs=max_epochs,
                                  train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
                                  model=model,
                                  loss_fn=loss_fn,
                                  optimizer=optimizer,
                                  regularizer=regularizer,
                                  early_stopper=early_stopper,
                                  logger=logger,
                                  trial=trial)
    
    return optim_score

#### PSO Constants

In [7]:
DYNAMIC_HPs = {
    'num_hidden_layer': [3, 3],
    'hidden_layer_n1_size': [0, 128],
    'hidden_layer_n2_size': [0, 128],
    'hidden_layer_n3_size': [0, 128],
    'learning_rate': [1e-4, 1e-1]
}

In [8]:
pso = PSO(objective_fn=objective, hps_bounds=DYNAMIC_HPs, num_particles=2, n_iter=2)

In [9]:
ATTRS = ('generation', 'particle_id', 'hp_num_hidden_layer', 'score', 'user_attrs_epochs', 'user_attrs_network', 'user_attr_accuracy', 'user_attr_precision', 'user_attr_recall', 'user_attr_f1', 'state', 'duration', 'hp_hidden_layer_n1_size', 'hp_hidden_layer_n2_size', 'hp_hidden_layer_n3_size', 'hp_learning_rate' 'datetime_start', 'datetime_complete')

In [10]:
DIRECTION = 'maximize'

### Run Optimization

In [11]:
pso_runner = PSORunner(path_csv=outputs_folder_path_csv, path_txt=outputs_folder_path_txt, session_num=session_num, n_jobs=-1, metric_to_follow='accuracy', attrs=None)

In [12]:
pso_runner(pso, 'PSO_Optimization')


Trial Gen n°1 - Particle n°0 - Epoch 1
-------------------------------
Intermediate Avg loss:   0.9578
Intermediate Accuracy:   68.9167%
Intermediate Precision:  68.4691%
Intermediate Recall:     67.8694%
Intermediate F1:         65.4875%

Intermediate Optimization Score (Gen n°1 - Particle: n°0): 42.250%


Trial Gen n°1 - Particle n°0 - Epoch 2
-------------------------------
Intermediate Avg loss:   1.0174
Intermediate Accuracy:   66.9750%
Intermediate Precision:  67.9743%
Intermediate Recall:     65.4963%
Intermediate F1:         64.2472%

Intermediate Optimization Score (Gen n°1 - Particle: n°0): 40.308%


Trial Gen n°1 - Particle n°0 - Epoch 3
-------------------------------
Intermediate Avg loss:   0.8579
Intermediate Accuracy:   69.4500%
Intermediate Precision:  71.9378%
Intermediate Recall:     68.9557%
Intermediate F1:         68.0704%

Intermediate Optimization Score (Gen n°1 - Particle: n°0): 42.783%


Trial Gen n°1 - Particle n°0 - Epoch 4
-------------------------------
I

,generation,particle_id,score,user_attrs_network,user_attrs_epochs,user_attrs_accuracy,user_attrs_precision,user_attrs_recall,user_attrs_f1,network,...,f1,state,hp_num_hidden_layer,hp_hidden_layer_n1_size,hp_hidden_layer_n2_size,hp_hidden_layer_n3_size,hp_learning_rate,duration,datetime_start,datetime_complete
0,1,0,0.434633,"[784, 61, 125, 70, 10]",8,0.7013,0.726770,0.696534,0.689325,"[784, 61, 125, 70, 10]",...,0.689325,COMPLETE,3.0,61.244611,124.658853,70.404256,0.062269,0 days 00:01:48.505928,2024-05-06 20:10:06.783775,2024-05-06 20:11:55.289703
1,1,1,0.709375,"[784, 116, 52, 51, 10]",12,0.9375,0.938736,0.936983,0.937424,"[784, 116, 52, 51, 10]",...,0.937424,COMPLETE,3.0,115.810013,51.778351,51.443358,0.036895,0 days 00:02:36.438379,2024-05-06 20:11:55.289703,2024-05-06 20:14:31.728082
2,2,0,0.313908,"[784, 93, 80, 62, 10]",6,0.5587,0.491000,0.549249,0.498385,"[784, 93, 80, 62, 10]",...,0.498385,COMPLETE,3.0,92.929221,79.757120,61.503277,0.100000,0 days 00:01:18.737296,2024-05-06 20:14:31.728082,2024-05-06 20:15:50.465378
3,2,1,0.086775,"[784, 116, 52, 51, 10]",8,0.3149,0.254384,0.314465,0.250214,"[784, 116, 52, 51, 10]",...,0.250214,COMPLETE,3.0,115.864564,51.888568,51.298799,0.100000,0 days 00:01:44.384193,2024-05-06 20:15:50.465378,2024-05-06 20:17:34.849571
